# Census.gov Reference Data

In [1]:
from datetime import datetime
from os import environ
import re
import urllib

import numpy as np
import openpyxl
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [ ]:
%%bash

pip3 list

## Counties (Census.gov)
- Counties are source from census.gov [gazetteer files](https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2019_Gazetteer/2019_Gaz_counties_national.zip)

In [ ]:
%%time

county_uri = 'https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2019_Gazetteer/2019_Gaz_counties_national.zip'

counties = (pd.read_table(county_uri,
                          dtype={'GEOID': str, 'ANSICODE': str})
           .rename(columns=lambda x: x.strip()))
counties.shape

In [ ]:
counties.dtypes

In [ ]:
counties['StateFIPS'] = counties.GEOID.str[:2]
counties['CountyFIPS'] = counties.GEOID.str[2:]
# counties[['CountyPrefix', 'CountySuffix']] = counties.NAME.str.rsplit(n=1, expand=True)
# counties['CountySuffix'] = counties['CountySuffix'].str.upper()

In [ ]:
county_pattern = re.compile(pattern=r'\sCounty$')

counties.loc[
    counties.NAME.str.contains(county_pattern)
    , ['CountyPrefix']] = counties.NAME.str.replace(county_pattern, repl='')

counties.loc[
    counties.NAME.str.contains(county_pattern)
    , ['CountySuffix']] = 'COUNTY'

counties.loc[counties.CountyPrefix.isna(), ['CountyPrefix']] = counties.NAME

In [ ]:
counties.loc[~counties.NAME.str.contains('County'), :]

## States/Regions (Census.gov)
- Census.gov's [States and regions](https://www2.census.gov/programs-surveys/popest/geographies/2019/state-geocodes-v2019.xlsx)

In [ ]:
st_uri = 'https://www2.census.gov/programs-surveys/popest/geographies/2019/state-geocodes-v2019.xlsx'

st = pd.read_excel(st_uri, engine='openpyxl', header=1, skiprows=4,
                   dtype={'Region': str, 'Division': str, 'State (FIPS)': str})

st_region = st.loc[st['Name'].str.endswith('Region'), 
                   ['Region','Name']]

st_division = st.loc[st['Name'].str.endswith('Division'), 
                   ['Division','Name']]

In [ ]:
st_final = (st.loc[st['State (FIPS)'] != '00', :]
            .merge(st_region, on='Region', how='left')
            .merge(st_division, on='Division', how='left')
            .rename(columns={
                'State (FIPS)':'StateFIPS', 'Name_x': 'StateName',
                'Name_y': 'RegionName', 'Name': 'DivisionName'
            })
            .merge(counties.loc[:, ['StateFIPS', 'USPS']], on='StateFIPS', how='left')
            .drop_duplicates()
            .reset_index(drop=True)
)

st_final.head()

## Places (Census.gov)
- Places are sourced from census.gov [gazetteer files](https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2019_Gazetteer/2019_Gaz_place_national.zip)

In [ ]:
%%time

places_uri = 'https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2019_Gazetteer/2019_Gaz_place_national.zip'

places = (pd.read_table(places_uri,
                        dtype={'GEOID': str, 'ANSICODE': str})
          .rename(columns=lambda x: x.strip()))
places.shape

In [ ]:
places.dtypes

### Regular Expression Data Wrangling

In [ ]:
%%time

places['StateFIPS'] = places.GEOID.str[:2]
places['PlaceFIPS'] = places.GEOID.str[2:]

places[['CLEAN_NAME']] = (
    places.NAME.str.replace(pat=' (balance)', repl='', regex=False)
)

erase = re.compile(pattern=r'\s[(].+[)]')
synonym = re.compile(pattern=r'[(](?P<in_syn>.+)[)]')

places.loc[
    places.CLEAN_NAME.str.contains(pat=erase)
    , ['CLEAN_NAME']] = places.CLEAN_NAME.str.replace(pat=erase, repl='')

places.loc[
    places.CLEAN_NAME.str.contains(pat=erase)
    , ['PlaceSynonym']] = places.CLEAN_NAME.str.extract(pat=synonym, expand=False)

g_pattern = r"""
    (?P<PlacePrefix>.+?)  # non-greedy
    \s
    (?P<PlaceSuffix>
        borough  # 21
        |city  # 25
        |metro\stownship  # 35 (undocumented)
        |municipality  # 37
        |town  # 43
        |village  # 47
        |city\sand\sborough  # 53
        |comunidad  # 55
        |zona\surbana  # 62
        |CDP  # 57
        |consolidated\sgovernment  # CG
        |metro\w*\sgovernment  # MG
        |urban\scounty  # UC
        |unified\sgovernment  # UG
    )$
"""

g_compiled = re.compile(g_pattern, flags=re.X)  # re.IGNORECASE

name_split = places.CLEAN_NAME.str.extract(g_compiled)

unity = places.join(name_split)

unity.loc[
    unity.PlacePrefix.isna()
    , ['PlacePrefix']] = unity.CLEAN_NAME

unity['PlacePrefix'] = unity['PlacePrefix'].str.strip()
unity['PlaceSuffix'] = unity['PlaceSuffix'].str.strip()
unity['PlaceSuffix'] = unity['PlaceSuffix'].str.upper()
unity.drop(['CLEAN_NAME'], axis=1, inplace=True)

g1 = (unity
      .groupby('PlaceSuffix', sort=True, dropna=False)
      .agg('count')
     )
g1[['USPS']]

In [ ]:
unity.loc[unity.PlacePrefix.str.contains('Odessa'), :]